In [1]:
import multiprocessing as mp
from glob import glob
import os
import subprocess as sp
import shutil
from time import time, sleep
from threading import Timer
import psutil

In [2]:
cd = os.getcwd() + '/'
wd = '{}/xf/runs/'.format(cd).replace('//', '/')

In [3]:
num_proc = 3

In [4]:
def get_dirs(wd):
    rd = glob(wd+'/*')
    r = {}
    for i in rd:
        r[i.replace(wd, '')] = {}
        r[i.replace(wd, '')]['runs'] = [j.replace(i, '') for j in glob(i + '/*.run')]
        r[i.replace(wd, '')]['starts'] = [j.replace(i, '') for j in glob(i + '/*.start')]
        r[i.replace(wd, '')]['xf'] = [j.replace(i, '') for j in glob(i + '/*.start')]
    return r

def make_ds(wd):
    r = get_dirs(wd)
    ds = []
    k = 0
    for i in r.keys():
        for j in r[i]['runs']:
            if j[:-4]+'.start' not in r[i]['starts']:
                ds.append([i, j, k])
                k +=1
    return ds

def kill_porc(proc_pid):
    process = psutil.Process(proc_pid)
    for proc in process.children(recursive=True):
        print(proc.pid)
        proc.kill()
    process.kill()
    return

def runs(oper):
    af = oper[0]
    rf = oper[1]
    kk = oper[2]
    
    if kk % 5 == 0: 
        # delete the extra xfoil files
        h = get_dirs(wd)
        for fold in h.keys():
            if (len(h[fold]['runs']) == len(h[fold]['runs'])) and os.path.exists(wd + '/' + fold):
                try:
                    os.remove(wd + '/' + fold + '/xfoil-6.97')
                    print('deleted.')
                except:
                    varr = 0
    
    cwd = '{}/{}/'.format(wd, af).replace('//', '/')
    if not os.path.exists('{}/xfoil-6.97'.format(cwd)):
        shutil.copy('{}/xfoil-6.97'.format(cwd.replace('/runs/{}/'.format(af), '')), cwd)
    
    cmd = 'cd {}; xfoil < {}'.format(cwd,rf[1:]).replace('//', '/')
    print(cmd)
    with open('{}/{}.start'.format(cwd,rf[:-4]), 'w') as fidd:
        fidd.write(str(time()))
    kill = lambda process:process.kill()
    prc = sp.Popen(cmd, preexec_fn=os.setsid, shell=True) 
    try:
        prc.wait(60)
    except:
        kill_porc(prc.pid)
    return

def main(wd, core_num=3):
    ds = make_ds(wd)
    p = mp.Pool(processes=core_num)
    p.imap(runs, ds)
    p.close()
    p.join()
    return

In [5]:
%%time
main(wd, core_num=2)

CPU times: user 13.2 ms, sys: 8.77 ms, total: 22 ms
Wall time: 119 ms
